<a href="https://colab.research.google.com/github/GuillaumeArp/Wild_Notebooks/blob/main/Pandas_automation_Guillaume_Arp.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Function to automatic merge and fill missing values



## Create the function
You have to create a function which take 2 arguments :
- argument 1 : DataFrame main
- argument 2 : DataFrame opinion

Your function will return a new DataFrame which is the left merge of both DataFrames, and which has no missing values (so your function has to fill them), add new columns (flag, MONTH and DAY), and OPINION should be translated. 

So you have to copy each useful code from the previous quests, and integrate it into a function. **This should achieve the same results as the previous quests.**

In [ ]:
import pandas as pd
import numpy as np
import plotly.express as px
import plotly.graph_objects as go

In [ ]:
link_main = "https://raw.githubusercontent.com/murpi/wilddata/master/quests/weather_main_2018.csv"
link_opinion = "https://raw.githubusercontent.com/murpi/wilddata/master/quests/weather_opinion_2018.csv"
df_main = pd.read_csv(link_main)
df_opinion = pd.read_csv(link_opinion)
print(df_main.head())
print(df_opinion.head())

         DATE  MAX_TEMPERATURE_C  ...  DEWPOINT_MAX_C  WINDTEMP_MAX_C
0  2018-01-01                 12  ...               8               7
1  2018-01-02                 13  ...              12               6
2  2018-01-03                 15  ...              13               7
3  2018-01-04                 14  ...              12              10
4  2018-01-05                 12  ...              10               7

[5 rows x 15 columns]
         date  WEATHER_CODE_EVENING  TOTAL_SNOW_MM  UV_INDEX  SUNHOUR OPINION
0  2018-01-01                   113              0         3      5.1     bad
1  2018-03-12                   119              0         2      8.8     bad
2  2018-03-09                   116              0         3     10.2     bad
3  2018-10-07                   122              0         1      5.6     bad
4  2018-06-18                   119              0         1     12.9     bad


In [ ]:
def translate(word):
  translation_dict = {
    "very bad": "très mauvais",
    "bad": "mauvais",
    "not good not bad": "ni bon ni mauvais",
    "good": "bon",
    "very good": "très bon"
    }
  if word in translation_dict:
    return translation_dict[word]
  else:
    return word

In [ ]:
def merge_and_fill(DataFrameMain, DataFrameOpinion):

  df = pd.merge(DataFrameMain, DataFrameOpinion, how="left", left_on='DATE', right_on='date')
  df.drop('date', axis=1, inplace=True)
  df['flag'] = df['OPINION'].isna()
  df['DATE'] = pd.to_datetime(df['DATE'])
  df['YEAR'] = df['DATE'].apply(lambda x: x.year)
  df['MONTH'] = df['DATE'].apply(lambda x: x.month)
  df['DAY'] = df['DATE'].apply(lambda x: x.day)
  df['WEATHER_CODE_EVENING'].fillna(round(np.mean(df['WEATHER_CODE_EVENING'])), inplace=True)
  df['TOTAL_SNOW_MM'].fillna(round(np.mean(df['TOTAL_SNOW_MM'])), inplace = True)
  df['UV_INDEX'].fillna(round(np.mean(df['UV_INDEX'])), inplace = True)
  df['SUNHOUR'].fillna(method = 'bfill', inplace = True)
  df['OPINION'].fillna('inconnu', inplace = True)
  df['OPINION'] = df['OPINION'].apply(translate)

  return df

In [ ]:
df2018 = merge_and_fill(df_main, df_opinion)
df2018

,DATE,MAX_TEMPERATURE_C,MIN_TEMPERATURE_C,WINDSPEED_MAX_KMH,TEMPERATURE_MORNING_C,TEMPERATURE_NOON_C,TEMPERATURE_EVENING_C,PRECIP_TOTAL_DAY_MM,HUMIDITY_MAX_PERCENT,VISIBILITY_AVG_KM,PRESSURE_MAX_MB,CLOUDCOVER_AVG_PERCENT,HEATINDEX_MAX_C,DEWPOINT_MAX_C,WINDTEMP_MAX_C,WEATHER_CODE_EVENING,TOTAL_SNOW_MM,UV_INDEX,SUNHOUR,OPINION,flag,YEAR,MONTH,DAY
0,2018-01-01,12,8,61,9,11,8,8.9,79,9.500,1018,41.750,12,8,7,113.0,0.0,3.0,5.1,mauvais,False,2018,1,1
1,2018-01-02,13,6,26,8,12,13,0.6,96,9.000,1020,87.875,13,12,6,122.0,0.0,3.0,3.3,mauvais,False,2018,1,2
2,2018-01-03,15,10,40,11,12,10,5.5,82,8.500,1017,91.500,15,13,7,122.0,0.0,3.0,3.3,mauvais,False,2018,1,3
3,2018-01-04,14,11,45,14,14,11,0.0,89,10.000,1011,90.125,14,12,10,116.0,0.0,3.0,3.3,mauvais,False,2018,1,4
4,2018-01-05,12,7,21,10,11,8,1.5,85,9.875,1005,62.375,12,10,7,116.0,0.0,3.0,6.9,mauvais,False,2018,1,5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
360,2018-12-27,7,2,6,2,6,5,0.0,85,10.000,1027,30.750,8,6,3,119.0,0.0,1.0,8.7,très mauvais,False,2018,12,27
361,2018-12-28,7,2,8,2,7,3,0.0,89,8.000,1035,18.750,8,4,4,113.0,0.0,1.0,8.7,très mauvais,False,2018,12,28
362,2018-12-29,7,1,6,1,6,4,0.0,94,7.000,1038,33.000,8,5,1,116.0,0.0,1.0,8.7,très mauvais,False,2018,12,29
363,2018-12-30,9,4,6,5,9,8,0.1,95,6.000,1038,70.375,10,9,7,143.0,0.0,1.0,3.3,très mauvais,False,2018,12,30


In [ ]:
df2018.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 365 entries, 0 to 364
Data columns (total 24 columns):
 #   Column                  Non-Null Count  Dtype         
---  ------                  --------------  -----         
 0   DATE                    365 non-null    datetime64[ns]
 1   MAX_TEMPERATURE_C       365 non-null    int64         
 2   MIN_TEMPERATURE_C       365 non-null    int64         
 3   WINDSPEED_MAX_KMH       365 non-null    int64         
 4   TEMPERATURE_MORNING_C   365 non-null    int64         
 5   TEMPERATURE_NOON_C      365 non-null    int64         
 6   TEMPERATURE_EVENING_C   365 non-null    int64         
 7   PRECIP_TOTAL_DAY_MM     365 non-null    float64       
 8   HUMIDITY_MAX_PERCENT    365 non-null    int64         
 9   VISIBILITY_AVG_KM       365 non-null    float64       
 10  PRESSURE_MAX_MB         365 non-null    int64         
 11  CLOUDCOVER_AVG_PERCENT  365 non-null    float64       
 12  HEATINDEX_MAX_C         365 non-null    int64     

## Execution with a loop
You will execute your function in a 9 rounds loop, once per year, and agregate this in a new big DataFrame with 24 columns and lots of rows.

In [ ]:
# Each year between 2010 and 2018 (included) has 2 sources : weather_main_YYYY.csv and weather_opinion_YYYY.csv
df = pd.DataFrame()

for year in range(2010, 2019):
  url_main = "https://raw.githubusercontent.com/murpi/wilddata/master/quests/weather_main_" + str(year) + ".csv"
  url_opinion = "https://raw.githubusercontent.com/murpi/wilddata/master/quests/weather_opinion_" + str(year) + ".csv"
  df_main = pd.read_csv(url_main)
  df_opinion = pd.read_csv(url_opinion)
  df_temp = merge_and_fill(df_main, df_opinion)
  df = pd.concat([df, df_temp])

df.reset_index(drop=True, inplace=True)
df.info()



<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3287 entries, 0 to 3286
Data columns (total 24 columns):
 #   Column                  Non-Null Count  Dtype         
---  ------                  --------------  -----         
 0   DATE                    3287 non-null   datetime64[ns]
 1   MAX_TEMPERATURE_C       3287 non-null   int64         
 2   MIN_TEMPERATURE_C       3287 non-null   int64         
 3   WINDSPEED_MAX_KMH       3287 non-null   int64         
 4   TEMPERATURE_MORNING_C   3287 non-null   int64         
 5   TEMPERATURE_NOON_C      3287 non-null   int64         
 6   TEMPERATURE_EVENING_C   3287 non-null   int64         
 7   PRECIP_TOTAL_DAY_MM     3287 non-null   float64       
 8   HUMIDITY_MAX_PERCENT    3287 non-null   int64         
 9   VISIBILITY_AVG_KM       3287 non-null   float64       
 10  PRESSURE_MAX_MB         3287 non-null   int64         
 11  CLOUDCOVER_AVG_PERCENT  3287 non-null   float64       
 12  HEATINDEX_MAX_C         3287 non-null   int64   

In [ ]:
df

,DATE,MAX_TEMPERATURE_C,MIN_TEMPERATURE_C,WINDSPEED_MAX_KMH,TEMPERATURE_MORNING_C,TEMPERATURE_NOON_C,TEMPERATURE_EVENING_C,PRECIP_TOTAL_DAY_MM,HUMIDITY_MAX_PERCENT,VISIBILITY_AVG_KM,PRESSURE_MAX_MB,CLOUDCOVER_AVG_PERCENT,HEATINDEX_MAX_C,DEWPOINT_MAX_C,WINDTEMP_MAX_C,WEATHER_CODE_EVENING,TOTAL_SNOW_MM,UV_INDEX,SUNHOUR,OPINION,flag,YEAR,MONTH,DAY
0,2010-01-01,5,2,19,2,5,3,0.0,86,10.000,1012,62.875,5,3,-1,119.0,0.0,1.0,5.1,très mauvais,False,2010,1,1
1,2010-01-02,5,-1,6,-1,4,1,0.0,78,10.000,1022,31.375,5,0,2,116.0,0.0,1.0,8.7,très mauvais,False,2010,1,2
2,2010-01-03,5,1,26,2,4,2,0.3,61,9.875,1021,65.500,5,0,-3,119.0,0.0,1.0,6.9,très mauvais,False,2010,1,3
3,2010-01-04,1,-4,25,-2,0,-3,0.0,75,10.000,1017,9.875,1,0,-9,113.0,0.0,1.0,8.7,très mauvais,False,2010,1,4
4,2010-01-05,2,-5,20,-5,1,-4,0.0,71,10.000,1012,8.000,3,0,-10,113.0,0.0,1.0,8.7,très mauvais,False,2010,1,5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3282,2018-12-27,7,2,6,2,6,5,0.0,85,10.000,1027,30.750,8,6,3,119.0,0.0,1.0,8.7,très mauvais,False,2018,12,27
3283,2018-12-28,7,2,8,2,7,3,0.0,89,8.000,1035,18.750,8,4,4,113.0,0.0,1.0,8.7,très mauvais,False,2018,12,28
3284,2018-12-29,7,1,6,1,6,4,0.0,94,7.000,1038,33.000,8,5,1,116.0,0.0,1.0,8.7,très mauvais,False,2018,12,29
3285,2018-12-30,9,4,6,5,9,8,0.1,95,6.000,1038,70.375,10,9,7,143.0,0.0,1.0,3.3,très mauvais,False,2018,12,30


## Add 2019 & 2020

In [ ]:
link2019 = "https://raw.githubusercontent.com/murpi/wilddata/master/quests/weather2019.csv"
link2020 = "https://raw.githubusercontent.com/murpi/wilddata/master/quests/weather2020.csv"

def clean_weather(df_temp):
  df_temp['flag'] = df_temp['OPINION'].isna()
  df_temp['DATE'] = pd.to_datetime(df_temp['DATE'])
  df_temp['YEAR'] = df_temp['DATE'].apply(lambda x: x.year)
  df_temp['SUNHOUR'].fillna(method = 'ffill', inplace = True)
  df_temp['OPINION'].fillna('inconnu', inplace = True)
  df_temp['OPINION'] = df_temp['OPINION'].apply(translate)

  return df_temp

for year in range(2019, 2021):
  url = "https://raw.githubusercontent.com/murpi/wilddata/master/quests/weather" + str(year) + ".csv"
  df_add = pd.read_csv(url)
  df_temp = clean_weather(df_add)
  df = pd.concat([df, df_temp])

df.reset_index(drop=True, inplace=True)
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4018 entries, 0 to 4017
Data columns (total 26 columns):
 #   Column                  Non-Null Count  Dtype         
---  ------                  --------------  -----         
 0   DATE                    4018 non-null   datetime64[ns]
 1   MAX_TEMPERATURE_C       4018 non-null   int64         
 2   MIN_TEMPERATURE_C       4018 non-null   int64         
 3   WINDSPEED_MAX_KMH       4018 non-null   int64         
 4   TEMPERATURE_MORNING_C   4018 non-null   int64         
 5   TEMPERATURE_NOON_C      4018 non-null   int64         
 6   TEMPERATURE_EVENING_C   4018 non-null   int64         
 7   PRECIP_TOTAL_DAY_MM     4018 non-null   float64       
 8   HUMIDITY_MAX_PERCENT    4018 non-null   int64         
 9   VISIBILITY_AVG_KM       4018 non-null   float64       
 10  PRESSURE_MAX_MB         4018 non-null   int64         
 11  CLOUDCOVER_AVG_PERCENT  4018 non-null   float64       
 12  HEATINDEX_MAX_C         4018 non-null   int64   

# Draw a scatterplot
With Date on X-Axis and Max Temperature on Y-axis

The result must have 11 seasonality for the 11 summers/winters alternance.




In [ ]:
fig = px.scatter(
    data_frame=df,
    x='DATE',
    y='MAX_TEMPERATURE_C',
    color='OPINION',
    template='plotly_dark',
    color_discrete_map={'très mauvais': 'red', 'inconnu': 'gray', 'mauvais': 'orange', 'ni bon ni mauvais': 'yellow', 'bon': 'lightgreen', 'très bon': 'green'},
    labels={'DATE': 'Date', 'MAX_TEMPERATURE_C': 'Max Temperature', 'OPINION': 'Opinion'},
    hover_name='OPINION',
    title='Max Temperature over 10 years sorted by opinion',
)

fig.show()


Find appropriate dataviz to answer to this question : 

Which year had the best weather? Are years very different or not ?


In [ ]:
fig = px.histogram(df, x='OPINION', facet_col='YEAR',
                   template='plotly_dark',
                   color='OPINION',
                   color_discrete_map={'très mauvais': 'red', 'inconnu': 'gray', 'mauvais': 'orange', 'ni bon ni mauvais': 'yellow', 'bon': 'lightgreen', 'très bon': 'green'},
                   labels={'YEAR': 'Année', 'OPINION': 'Opinion'})
fig.update_layout(title='Distribution des opinions sur les 10 dernières années')
fig.show()

Years are not very different overall, but we can clearly see on the scatterplot that temperatures tend to get hotter over the years. The histogram shows that there are some years with a greater number than usual of very bad or bad opinions, but the mean seems to be quite stable.

If we had to pick one year that had the best weather, 2014 would be a good candidate due to the very low number of bad opinions, and the high number of good opinions. We should keep in mind though that this year laso had a high number of missing values, which could explain why there are few very bad and bad reviews.